In [ ]:
!pip install -q groq

In [ ]:
from groq import Groq

# your Groq API key
client = Groq(
    api_key='',
)

# your fakeyou email and password
fy_email = ''
fy_password = ''



| Supported Characters 	| FakeYou Voice IDs                	|
|----------------------	|----------------------------------	|
| rick sanchez         	| weight_0f762jdzgsy1dhpb86qxy4ssm 	|
| barack obama         	| weight_8f3r8hwt0j58j68ayzzfgsh8h 	|



In [ ]:
def scriptGen(person1, person2):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"Write the script for a 5 min podcast between {person1} and {person2}, showcasing their personalities. dialogues only without laughs, excitedly, pauses. no bold.",
          }
      ],
      model="llama3-70b-8192",
  )
  print(chat_completion.choices[0].message.content)
  script = chat_completion.choices[0].message.content
  return script

#scriptGen("rick sanchez, barack obama")

In [ ]:
def parse_dialogue(dialogue):
    lines = dialogue.split('\n\n')
    dialogues = []
    current_character = None
    for line in lines:
        if line.startswith('Rick Sanchez'):
            current_character = 'Rick Sanchez'
            dialogues.append({'Character': current_character, 'Dialogue': line.split(':', 1)[1].strip(), 'VoiceID': "weight_0f762jdzgsy1dhpb86qxy4ssm"})
        elif line.startswith('Barack Obama'):
            current_character = 'Barack Obama'
            dialogues.append({'Character': current_character, 'Dialogue': line.split(':', 1)[1].strip(), 'VoiceID': "weight_8f3r8hwt0j58j68ayzzfgsh8h"})
        elif current_character:
            dialogues[-1]['Dialogue'] += ' ' + line.strip()
    return dialogues

#print(parse_dialogue(scriptGen("rick sanchez, barack obama")))

In [ ]:
!pip install fakeyou

In [ ]:
import fakeyou
import time

# Initialize the fakeyou client
fy = fakeyou.FakeYou()

# Call the login method with email and password
fy.login(fy_email, fy_password)

# Parse the dialogues
dialogues = parse_dialogue(scriptGen("rick sanchez, barack obama"))
var = -1
for idx, dialogue in enumerate(dialogues):
    text = dialogue['Dialogue']
    file_name = f"dialogue_{idx}.wav"
    voice_id = dialogue['VoiceID']
    fy.say(text, voice_id).save(file_name)
    print(f"Saved dialogue {idx} as {file_name}")
    var += 1
    time.sleep(30)

In [ ]:
import wave

def merge_wavs(output_file, input_files):

  with wave.open(output_file, 'wb') as output:
    with wave.open(input_files[0], 'rb') as input:
      output.setparams(input.getparams())
    for input_file in input_files:
      with wave.open(input_file, 'rb') as input:
        output.writeframes(input.readframes(input.getnframes()))

# Merge the WAV files
merge_wavs("podcast.wav", [f"dialogue_{i}.wav" for i in range(int(var))])